# Info
Purpose:

    To Calculate aerosol optical properties for aerosol above cloud reitrewvals using MODIS and VIIRS
    Using the wavelengths: 
    0.47, 0.55, 0.67, 0.86, 1.24, 2.1µm
    
    - Using the retrieved size distributions
    - retrieval results of refractive index (imaginary and real) at wavelengths: 400, 500, 675, 870, 995 nm


Output:

    Figure and save files

Keywords:

    none

Dependencies:

    - load_utils.py
    - matplotlib
    - numpy
    - Sp_parameters
    - write_utils
    - path_utils
    - libradtran

Needed Files:
  - netcdf of aeroinv from ORACLES
  - ...

Modification History:

    Written: Samuel LeBlanc, Santa Cruz, CA, 2022-09-12
    Modified:


# Prepare python environment

In [3]:
import numpy as np
import Sp_parameters as Sp
import load_utils as lu
import write_utils as wu
from path_utils import getpath
import hdf5storage as hs
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib notebook
import os


In [7]:
name = 'sunsat_ORACLES2016'
vv = 'v1'
fp = getpath(name)


Created new path: /data/sunsat/ORACLES_2016/ under the name: sunsat_ORACLES2016
Return path named:  sunsat_ORACLES2016 /data/sunsat/ORACLES_2016/


In [17]:
fp_lib = getpath('uvspec_bin')

Return path named:  uvspec_bin /home/sam/libradtran/libRadtran-2.0.2/bin/


In [27]:
fp_rtm0 = getpath('rtm')
fp_rtm = fp_rtm0 +'TASNPP_mie/'

Return path named:  rtm /scratch/rtm/


In [28]:
if not os.path.exists(fp_rtm): 
    os.mkdir(fp_rtm)

# Load files

In [8]:
f = fp + 'data_archival/AEROINV_nc_R0/NC_FORMAT_NETCDF4_CLASSIC/4STAR-aeroinv_P3_2016_R0.nc'

In [9]:
ae,ae_dict = lu.load_netcdf(f,everything=True)

Reading file: /data/sunsat/ORACLES_2016/data_archival/AEROINV_nc_R0/NC_FORMAT_NETCDF4_CLASSIC/4STAR-aeroinv_P3_2016_R0.nc
Outputting the Data subdatasets:
0: base_time
1: time_offset
2: time
3: wavelength
4: radius
5: psd
6: QA_level
7: sphericity
8: sphericity_err
9: PF_angle
10: sca_angle
11: n_real
12: n_imag
13: SSA
14: AOD_fit_total
15: AOD_fit_fine
16: AOD_fit_coarse
17: AOD_meas
18: AAOD
19: TOD_meas
20: TOD_fit
21: TOD_meas_minus_fit
22: AGOD
23: sfc_alb
24: g_total
25: g_fine
26: g_coarse
27: PF_total
28: PF_fine
29: PF_coarse
30: normalized_sky_radiance
31: normalized_sky_radiance_fit
32: sky_radiance_fit_error
33: scan_tag
34: scan_type
35: Lat
36: Lon
37: GPS_Altitude
38: Heading
39: Pitch
40: Roll
41: GPS_Altitude_std
42: Heading_std
43: Pitch_std
44: Roll_std
45: T_static
46: P_static
47: RH
48: SAZ
49: SZA
50: SEL
51: m_ray
52: PWV
[b'base_time', b'time_offset', b'time', b'wavelength', b'radius', b'psd', b'QA_level', b'sphericity', b'sphericity_err', b'PF_angle', b'sca_a

In [14]:
len(ae[b'time'])

86

# Run through each retrieval and make input files

## Prep functions for printing size distribution

From the libradtran documentation for mie size distribution file:  
>Specify a two column file, r [micron], dn(r)/dr, which describes a size distribution

In [34]:
def print_size_dist_file(fname,r,dnr):
    with open(fname,'w') as f:
        for ir,rr in list(enumerate(r)):
            f.write('{:3.10f} {:3.10f}\n'.format(rr,dnr[ir]))

In [20]:
ae_dict[b'radius']

<class 'netCDF4._netCDF4.Variable'>
float64 radius(radius)
    long_name: particle radius
    units: um
unlimited dimensions: 
current shape = (22,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [21]:
ae_dict[b'psd']

<class 'netCDF4._netCDF4.Variable'>
float64 psd(time, radius)
    long_name: particle size distribution
    units: dV/dlnR
    source: retrieval
unlimited dimensions: time
current shape = (86, 22)
filling on, default _FillValue of 9.969209968386869e+36 used

In [25]:
def convert_dvlnr_to_dndr(psd,r):
     # All about that conversion from the volume size distribution of dV(r)/dln(r) to number size distribution dN(r)/dr
    Nr = psd/(4.0*np.pi/3.0)
    for i,rr in list(enumerate(r)):
        Nr[i] = Nr[i]/rr**4.0
    return Nr

In [37]:
print_size_dist_file(fp_rtm+'mie_tester.psd',ae[b'radius'],convert_dvlnr_to_dndr(ae[b'psd'][0,:],ae[b'radius']))

## Prep function for printing index of refraction